### 적절한 형태소 분석기 찾기

In [ ]:
# 우수한 형태소 분석기 추출 => 결과 hanNouns


#-- 기본 개념 --#
# morphs 형태소 단위
# nouns 단어
# pos 텍스트에 품사정보 포함

#-- Hannanum --#
hannanum = Hannanum()
hanMorphs = hannanum.morphs(doc)
hanNouns = hannanum.nouns(doc)
hanPos = hannanum.pos(doc)

#-- kkma --#
kkma = Kkma()
kkmaMorphs = kkma.morphs(doc)
kkmaNouns = kkma.nouns(doc)
kkmaPos = kkma.pos(doc)

#-- Okt --#
okt = Okt()
oktMorphs = okt.morphs(doc)
oktNouns = okt.nouns(doc)
oktPos = okt.pos(doc)


model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
finish = {}
for num, testType in enumerate([hanMorphs,hanNouns,kkmaMorphs,kkmaNouns,oktMorphs,oktNouns]) :
    words = ' '.join(testType)
    vect = CountVectorizer(ngram_range=(1,2))
    count = vect.fit([words])

    candidate = count.get_feature_names_out()
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidate)

    distances = cosine_similarity(doc_embedding,candidate_embeddings)
    # result = candidate[distances.argsort()[0][-5:]]
    result = mmr(doc_embedding, candidate_embeddings, candidate, top_n=50, diversity=0.2)
    finish[num] = result


### 최종선택 hanNouns
### 최종선택 hanNouns
### 최종선택 hanNouns

### 웹 크롤러 예전버전

In [ ]:
def kyoboExtract(ISBN:str) -> dict : 
    kyoboUrl = f'http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}'
    kyoboHtml = requests.get(kyoboUrl)
    kyoboSoup = BeautifulSoup(kyoboHtml.content)

    #-- title --#
    bookTitle : str = kyoboSoup.h1.strong.string.strip()

    #-- book Intro --#
    bookIntro : str = kyoboSoup.find('div',{'class':'box_detail_article'}).string.strip()

    #-- author Intro --#
    authorIntro : str = kyoboSoup.find('div',{'class' : 'box_detail_article'}).string.strip()

    #-- keywords --#
    keywords = kyoboSoup.find('div',{'class':'tag_list'}).find_all('a')
    keywords : list = [k.string.replace('#','') for k in keywords]

    #-- TOC --#
    toc = kyoboSoup.find_all('div',{'class' : 'box_detail_article'})[3]
    toc = re.sub('<.*?>|\\t','',str(toc))
    toc = re.sub('\\n',' ',toc)
    toc :str = re.sub('\\r',',',toc)
    toc = re.sub('\d[.]|\d|[,]\s*','',toc) 
    
    return dict(bookTitle=bookTitle,bookIntro=bookIntro,authorIntro=authorIntro,toc=toc,keywords=keywords)


kyoboExtract(9791160505979)


### lib info extraction

In [2]:
import pandas as pd
a = pd.read_excel('./data/lib_info.xlsx',)
a = a[6:]
a.columns = a.iloc[0]
a = a[1:]
a = a[['도서관명','도서관코드']].reset_index(drop=True)
a.columns= ['libName','libCode']
a.to_csv('./data/lib_info.csv',encoding='CP949')

/Users/yangwoolee/Library/Python/3.8/lib/python/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
### Response로 정보 얻는건 불가능
import requests
import ast
lib_info = pd.read_csv('./data/lib_info.csv',encoding='CP949',index_col=0)
lib_info[lib_info['libName'].str.contains('양천')]

key = '7123eacb2744a02faca2508a82304c3bf154bf0b285da35c2faa2b8498b09872'
libCode=lib_info['libCode'][104]
start='2018-01-01'
end='2022-06-30'
response = requests.get(f'http://data4library.kr/api/itemSrch?authKey={key}&libCode={libCode}&startDt={start}&endDt={end}&format=JSON&pageSize=100')



a = ast.literal_eval(str(response.content.decode('utf-8')))

a

